In [594]:
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import ray
import parallel, propagation, read, simulation

In [595]:
import importlib
importlib.reload(simulation)
#ray.init()

<module 'simulation' from '/home/sarming/tmp/propagation/simulation.py'>

## Loading the anonymized follower graphs
Right now we read in the adjacency lists using NetworkX. This builds a dense representation which needs quadratric time and space.
So we only want to do this only once and store it as a sparse matrix together with node labels.

In [454]:
graph = read.adjlist('data/anonymized_outer_graph_neos_20200311.adjlist')
A = nx.to_scipy_sparse_matrix(graph)
node_labels = graph.nodes()
read.save_labelled_graph('data/outer_neos.npz', A, node_labels, compressed=False)

Or, alternatively, in one line:

In [ ]:
read.adjlist('data/anonymized_outer_graph_neos_20200311.adjlist', save_as='data/outer_neos.npz')

For the future we can now use the much faster:

In [3]:
A, node_labels = read.labelled_graph('data/outer_neos.npz')

## Basic message propagation
The fundamental function of our model is `edge_propagate`.
In a single call node `source` sends a message to all its followers, who with (uniform) edge probability `p` retweet it in turn.
At every level, `p` gets smaller by multiplication with a discount factor `discount` $< 1$.
If a node receives a message a second time, it discards it, irrespective of if it retweeted it or not.
This continues until either maximimum depth `depth` is reached or `max_nodes` have retweeted the message.

The return value is the number of nodes that resent/retweeted the message.

The concrete values below are not realistic for our twitter use case (`p` is much too high and `discount` to low), but allow to play around and observe the stochastic nature of the call. 

In [547]:
propagation.edge_propagate(A, source=0, p=0.5, discount=0.2, depth=10, max_nodes=10000)

3300

We can run several propagations using the function `simulate`.
It takes a list of source nodes and runs `samples` many propagations for each start node.

It returns the mean number of retweets (ie. the expected number of retweets) per tweet, and the fraction of tweets that are retweeted at least once (ie. the retweet probability).
These are the two main statistics we use to compare our model to the data.

In [585]:
propagation.simulate(A, sources=[0,1,2], p=0.01, discount=0.9, depth=10, max_nodes=1000, samples=1000)

(2.1483333333333334, 0.024333333333333332)

Passing `return_stats=False` the `simulate` function yields the complete results.
In the example below one can observe the crucial role the outdegree of the source node plays: $\delta^-(0)=7$, $\delta^-(1)=0$, $\delta^-(3)=232$

In [590]:
for source in propagation.simulate(A, sources=[0,1,3], p=0.01, discount=0.9, depth=10, max_nodes=10000, samples=20, return_stats=False):
    print(list(source))

[0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[6, 3, 3, 2, 1643, 0, 1, 1, 398, 3, 6, 5483, 180, 1, 7, 13, 1615, 0, 5, 0]


## Tweet data
For every source tweet in our dataset we extract several features of both the author and the tweet and the number of retweets.
The function `read.tweets` reads such an `.csv` file, aggregates selected features we are interested in into a single `author_feature` and a  single `tweet_feature` (in this case, since we only consider binary features, a simple binary string) and  also translates the author id into a node id (`source`)


In [592]:
tweets = read.tweets('data/authors_tweets_features_neos.csv', node_labels)
tweets

,source,author_feature,tweet_feature,retweets
0,15193,0010,1010,0
1,83877,0001,1010,2
2,39676,0010,1100,0
3,912617,0001,1110,6
4,13477,0101,0000,21
...,...,...,...,...
7451,48182,1011,0010,5
7452,886098,0000,1100,1
7453,39347,0000,0010,14
7454,12281,0010,0000,0


From this we can now calculate statistics for each feature combination.

In [596]:
simulation.tweet_statistics(tweets)

tweets  retweet_probability  mean_retweets  \
author_feature tweet_feature                                               
0000           0000              275             0.247273       1.094545   
               0001               17             0.235294       1.823529   
               0010              221             0.257919       1.687783   
               0011               32             0.343750       0.968750   
               0100               35             0.114286       1.085714   
...                              ...                  ...            ...   
1101           0111               12             1.000000       8.833333   
               1000               11             0.909091       6.090909   
               1010              149             0.959732       4.281879   
               1011               58             0.948276       5.931034   
               1100               12             0.583333       1.083333   

                              median_retweets  max_retweets  
author_feature tweet_feature                                 
0000           0000                       0.0            86  
               0001                       0.0            21  
               0010                       0.0            23  
               0011                       0.0             7  
               0100                       0.0            28  
...                                       ...           ...  
1101           0111                       4.0            33  
               1000                       4.0            26  
               1010                       3.0            30  
               1011                       4.0            44  
               1100                       1.0             4  

[100 rows x 5 columns]

## Finding the simulation parameters


In [3]:
datadir = '/Users/ian/Nextcloud'
# datadir = '/home/sarming'
#read.adjlist(f'{datadir}/anonymized_outer_graph_neos_20200311.adjlist',
#              save_as=f'{datadir}/outer_neos.npz')
sim = propagation.Simulation.from_files(f'{datadir}/outer_neos.npz', f'{datadir}/authors_tweets_features_neos.csv' )
features = sim.features
params = sim.params
stats = sim.stats

AttributeError: module 'propagation' has no attribute 'Simulation'

In [ ]:
stats.max_retweets.max()

In [ ]:
np.array_split(features[0:10],32)


In [ ]:
import itertools, operator
stats.groupby('author_feature')['sources'].apply(lambda ls: set(filter(pd.notna,itertools.chain(*ls))))
#agg(operator.concat)

In [ ]:
sim.sources[('01000','1')]

In [ ]:
type(stats.loc[('0000','0000'),'tweets'])

In [ ]:
stats.info()

In [ ]:
stats.loc[('0000','0000')]

In [ ]:

sim.search_parameters(1,0.5)

In [ ]:
sim.search_parameters(200,0.0001)

In [ ]:
features.to_csv('features_200_0.0001.csv')

In [ ]:
csv=pd.read_csv(f'{datadir}/authors_tweets_features_neos.csv' )
csv['author_feature']=list(zip(csv.))

In [ ]:
list(params)

In [ ]:
[(x) for x,y in params]

In [ ]:
p=ray.get(params)

In [ ]:
 def helper(feature):
        sim.search_parameters(100,0.00001)
    sample_calls = [(A, coef, random_vector(n)) for _ in range(num_samples)]
s = sum(pool.starmap(chebyshev_sample, sample_calls))

In [ ]:
list(sim.simulate(sim.sample_feature(10)))

In [ ]:
sim.sample_source('0000',size=None)

In [ ]:
tweets.dropna().groupby('author_feature')['source'].unique()

In [ ]:
sim.sample_source('0000',1)[0]
# features.loc[('0000','0011')]

In [ ]:
af = features.groupby('author_feature')['tweets'].sum()
af /= af.sum()
np.random.choice(af.index,size=1,p=af)


In [ ]:
with pd.option_context("display.max_rows", 1000): display(sim.features)

In [ ]:
sim.sample_source('0000',size=None)

In [ ]:
tweets.dropna().groupby('author_feature')['source'].unique()

In [ ]:
sim.sample_source('0000',1)[0]
# features.loc[('0000','0011')]

In [ ]:
af = features.groupby('author_feature')['tweets'].sum()
af /= af.sum()
np.random.choice(af.index,size=1,p=af)


In [ ]:
with pd.option_context("display.max_rows", 1000): display(sim.features)